In [1]:
from DiSuQ.Torch import models
from torch import tensor,stack
from numpy import arange,linspace,meshgrid,array,log,argsort
from DiSuQ.utils import plotCompare,plotHeatmap,plotBox
from DiSuQ.Torch.optimization import uniformParameters,truncNormalParameters,initializationSequential
from DiSuQ.Torch import optimization
from DiSuQ.Torch.optimization import lossDegeneracy,Adam
from DiSuQ.Torch.components import indE,capE
from torch import set_num_threads
set_num_threads(32)

In [2]:
L_ = indE(1e-7); L0 = indE(5*1e-9); print('Inductance Bound(GHz):',L_,L0)
C0 = capE(5*1e-16); C_ = capE(1e-14*5) ; print('Capacitance Bound(GHz):',C_,C0)
CJ0 = capE(5*1e-16/4); CJ_ = capE(1e-14/2) ; print('Shunt Bound(GHz):',CJ_,CJ0)
J0 = 25. ; J_ = 5. ; print('Junction Bound(GHz):',J_,J0)
# components['Jx'].J0 = J0 ; components['Jy'].J0 = J0

Inductance Bound(GHz): 1.6346151294983178 32.69230258996635
Capacitance Bound(GHz): 0.38740458615415185 38.740458615415186
Shunt Bound(GHz): 3.8740458615415188 154.96183446166074
Junction Bound(GHz): 5.0 25.0


In [3]:
basis = {'O':[4,4,4],'J':[5,5],'I':[]} ; rep = 'K'
flux_point = ['La','Lc']
circuit = models.prismon(basis,Ej=10.,Ec=5.,El=10.,EcJ=50.,sparse=False,symmetry=True,
                         _L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0))
static = circuit.circuitState()

In [4]:
lossFunction = lossDegeneracy

In [5]:
flux_profile = [{'Lx':tensor(.5),'Ly':tensor(0.0)}]

In [6]:
Algo = ['Nelder-Mead','LBFGS']

In [7]:
def multiInitSearch(optimizer,initials,subspace):
    NelMea,LBFGS = [],[]
    for index,parameter in enumerate(initials):
        print(optimizer.circuitState())
        print(index,parameter)
        optimizer.circuit.initialization(parameter)  
        optimizer.parameters,optimizer.IDs = optimizer.circuitParameters()
        LBFGS.append(optimizer.minimization(lossFunction,flux_profile,subspace=subspace,
                                                   method='L-BFGS-B',options=dict(ftol=1e-6,maxiter=5)))

        optimizer.circuit.initialization(parameter)
        optimizer.parameters,optimizer.IDs = optimizer.circuitParameters()
        NelMea.append(optimizer.minimization(lossFunction,flux_profile,subspace=subspace,
                                                 method='Nelder-Mead',options=dict(fatol=1e-6,maxiter=30)))

#         optimizer.circuit.initialization(parameter)
#         optimizer.parameters,optimizer.IDs = optimizer.circuitParameters()
#         lrBFGS.append(optimizer.optimizationLBFGS(lossFunction,flux_profile,iterations=iterations,lr=.0005))        

    return NelMea,LBFGS       

In [8]:
def optimizationAnalysis(init,subspace,Search,success=1.):
    Loss,Success,Paths = [],[],[]
    for index,(init,(dLogs,dParams,dCircuit)) in enumerate(zip(init,Search)):
        Paths.append(dCircuit[subspace].to_numpy())
        loss = dLogs['loss'].to_numpy()
        Loss.append(loss[-1])
        if loss[-1] < success:
            Success.append(len(loss))
    return Paths,Loss,Success

In [9]:
def analysisPlotting(Optimization):
    paths = dict()
    losse = dict()
    for algo,(Paths,Loss,Success) in zip(Algo,Optimization):
        indices = argsort(Loss)[0]
        paths[algo] = Paths[indices]
        #for idx in indices:
        #    paths[algo+'-'+str(idx)] = Paths[idx]
        losse[algo] = Loss
    return paths,losse

In [10]:
def lossScapeBounds(paths):
    Paths = []
    for algo,path in paths.items():
        Paths.append(path)
    Paths = vstack(Paths)
    Paths = Paths[:, ~isnan(Paths).any(axis=0)]
    return Paths.min(0),Paths.max(0)

In [11]:
N = 5; subspace = ['La','Ca','Ja','CJa']
#initials = uniformParameters(circuit,subspace,N)
initials = truncNormalParameters(circuit,subspace,N=10,var=1)
len(initials)

10

In [12]:
optimizer = optimization.OrderingOptimization(circuit,representation=rep)

In [ ]:
Optimization = multiInitSearch(optimizer,initials,subspace)
NelMea,LBFGS = Optimization

{'La': 10.0, 'Ca': 5.0, 'Ja': 10.0, 'CJa': 50.0, 'Lb': 10.0, 'Cb': 5.0, 'Jb': 10.0, 'CJb': 50.0, 'Lc': 10.0, 'Cc': 5.0, 'Jc': 10.0, 'CJc': 50.0}
0 {'La': 9.876784197150727, 'Ca': 4.858182091163184, 'Ja': 9.055902049707555, 'CJa': 49.73656067513402, 'Lb': 10.0, 'Cb': 5.0, 'Jb': 10.0, 'CJb': 50.0, 'Lc': 10.0, 'Cc': 5.0, 'Jc': 10.0, 'CJc': 50.0}
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -1.01119D+00    |proj g|=  1.14625D+00

At iterate    1    f= -1.35116D+00    |proj g|=  1.25029D+00
  ys=-1.590E-01  -gs= 1.584E+00 BFGS update SKIPPED

At iterate    2    f= -1.67463D+00    |proj g|=  7.63982D-01


/Users/chishti/JupyterHub/DiSuQ/Torch/components.py:67: RuntimeWarning:

divide by zero encountered in scalar divide




At iterate    3    f= -5.38859D+00    |proj g|=  2.28443D+01

At iterate    4    f= -5.71653D+00    |proj g|=  4.08321D+00

At iterate    5    f= -5.75079D+00    |proj g|=  7.47657D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4      5     16      5     1     0   7.477D-01  -5.751D+00
  F =  -5.7507939338684082     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 


/usr/local/lib/python3.11/site-packages/scipy/optimize/_minimize.py:549: RuntimeWarning:

Method Nelder-Mead does not use gradient information (jac).



In [ ]:
iterations = 30

In [ ]:
plotCompare(arange(iterations),{'Nelder-Mead':NelMea[0][0]['D'],'LBFGS':LBFGS[0][0]['D']},'Degeneracy','iterations',export='pdf',size=(600,800))

In [ ]:
plotCompare(arange(iterations),{'Nelder-Mead':NelMea[0][0]['E10'],'LBFGS':LBFGS[0][0]['E10']},'Ist Excitation','iterations',export='pdf',size=(600,800))

In [ ]:
Result = [optimizationAnalysis(initials,subspace,algo) for algo in Optimization]
paths,losse = analysisPlotting(Result)

In [ ]:
plotBox(losse,'Loss Distribution',export='pdf',size=(600,800))

In [35]:
losse['LBFGS']

[-5.750793933868408,
 -5.827541828155518,
 -10.916869163513184,
 -5.783031463623047,
 -5.763093948364258,
 -5.740583419799805,
 -5.70994234085083,
 -5.771575450897217,
 -5.731040000915527,
 -8.765881538391113]

### spectrum - optimal circuits

In [ ]:
Nx,Ny  = 51,1
PhiX = linspace(0,1,Nx,endpoint=True)
PhiY = linspace(0,1,Ny,endpoint=True)
flux_range = meshgrid(PhiX,PhiY)
flux_range = array((flux_range[0].flatten(),flux_range[1].flatten())).T
flux_manifold = [flux for flux in tensor(flux_range)]

In [ ]:
Degeneracy,plots = dict(),dict()
for algo,path in paths.items():
    parameters = dict(zip(subspace,path[-1]))
    static.update(parameters)
    circuit.initialization(static)
    H_LC = circuit.kermanHamiltonianLC()
    H_J = circuit.kermanHamiltonianJosephson
    E0,(Ex1,Ex2) = circuit.spectrumManifold(flux_point,flux_manifold,H_LC,H_J,excitation=[1,2],grad=True)
    #Ex2 = Ex2.detach().numpy()
    #Ex1 = Ex1.detach().numpy()
    degeneracy = log(Ex2/Ex1)
    Degeneracy[algo]=degeneracy.detach().numpy()
    plots[algo+'-E10']=Ex1.detach().numpy()
    plots[algo+'-E20']=Ex2.detach().numpy()
    
#     optimizer.circuit.initialization(static)
#     Spectrum = [optimizer.spectrumOrdered(flux) for flux in flux_profile]
#     spectrum = stack([spectrum[:3] for spectrum,state in Spectrum]).detach().numpy()
#     Ex1 = spectrum[:,1]-spectrum[:,0]
#     Ex2 = spectrum[:,2]-spectrum[:,0]
#     plots[algo+'-e10']=Ex1
#     plots[algo+'-e20']=Ex2

In [ ]:
plotCompare(flux_range[:,0],Degeneracy,'Degeneracy-Optimal Circuits','flux profile',export='pdf',size=(600,800))

In [ ]:
plotCompare(flux_range[:,0],plots,'Spectrum-Optimal Circuits','flux profile','energy(GHz)',export='pdf',size=(600,800))

In [ ]:
paths

In [32]:
paths['Nelder-Mead']

array([[ 9.65739727,  4.64417458, 11.06699562, 49.87176514],
       [10.20062542,  4.67181873, 10.16780186, 48.00157547],
       [10.20062542,  4.67181873, 10.16780186, 48.00157547],
       [10.17799187,  5.07611084, 10.36666203, 44.96251297],
       [10.17799187,  5.07611084, 10.36666203, 44.96251297],
       [10.57126617,  5.32188225,  8.70985508, 41.66045761],
       [10.57126617,  5.32188225,  8.70985508, 41.66045761],
       [10.73218346,  6.21755552,  7.93589497, 37.55115128],
       [10.73218346,  6.21755552,  7.93589497, 37.55115128],
       [12.21700001,  6.96643162,  5.        , 29.14579201],
       [12.21700001,  6.96643162,  5.        , 29.14579201],
       [12.30460167,  7.69669151,  5.        , 23.98355103],
       [12.30460167,  7.69669151,  5.        , 23.98355103],
       [12.30460167,  7.69669151,  5.        , 23.98355103],
       [12.30460167,  7.69669151,  5.        , 23.98355103],
       [12.30460167,  7.69669151,  5.        , 23.98355103],
       [12.30460167,  7.

In [33]:
paths['LBFGS']

array([[ 9.63800335,  5.5430851 , 10.79927444, 49.42588425],
       [ 8.66411304, 31.02913094,  5.34441519, 38.58588791],
       [ 1.63461518, 26.36903   ,  5.68188429, 59.55873871],
       [ 1.63461518, 19.01948547,  7.59125757, 87.64173126]])

In [34]:
circuit.circuitState()

{'La': 1.634615182876587,
 'Ca': 19.019487380981445,
 'Ja': 7.591257572174072,
 'CJa': 87.64173126220703,
 'Lb': 1.634615182876587,
 'Cb': 19.019487380981445,
 'Jb': 7.591257572174072,
 'CJb': 87.64173126220703,
 'Lc': 1.634615182876587,
 'Cc': 19.019487380981445,
 'Jc': 7.591257572174072,
 'CJc': 87.64173126220703}

### Charge-Flux susceptibility

In [ ]:
# for algo,path in paths.items():
#     parameters = dict(zip(subspace,path[-1]))
#     static.update(parameters)
#     circuit.initialization(static)
#     H_LC = circuit.kermanHamiltonianLC()
#     H_J = circuit.kermanHamiltonianJosephson
#     E0,(Ex1,Ex2) = circuit.spectrumManifold(flux_point,flux_manifold,H_LC,H_J,excitation=[1,2],grad=True)
#     Ex2 = Ex2.detach().numpy()
#     Ex1 = Ex1.detach().numpy()
#     degeneracy = log(Ex2/Ex1)
#     Degeneracy[algo]=degeneracy
#     plots[algo+'-charge']=Ex1
#     plots[algo+'-flux']=Ex2

### Wavefunction